In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [35]:
# Load and configure webdriver.
options = Options()
# Stop browser windows from actually popping up.
options.add_argument('--headless')
# Install a browser for use by Selenium.
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [36]:
# Navigate to US Chess tournament search page.
USCF_URL = 'http://www.uschess.org/datapage/events-rated.php'
uscf_events_page = driver.get(USCF_URL)

In [37]:
# Select a month on the page.
date_to_visit = '11/2022'
date_search_box = driver.find_element('name', 'month')
date_search_box.send_keys(date_to_visit)

# Select CA as the State Code, which is where all 
# chess.com USCF tournaments are registered.
state_search_box = driver.find_element('name', 'states')
state_search_box.send_keys('CA')
state_search_box.submit()

driver.save_screenshot('test_screenshot.png')

True

In [51]:
# Now that we have navigated to a results page that displays
# a table of tournaments, we can select table elements.

# Isolate only the data in the table body.
table_body = driver.find_elements(By.TAG_NAME, 'tbody')[2]
table_body_row = table_body.find_elements(By.TAG_NAME, 'tr')
table_row = table_body_row[4].find_elements(By.TAG_NAME, 'td')

# Get link to tournament page
print(table_row[0].find_element(By.TAG_NAME, 'a').get_attribute('href'))

print(table_row[1].text)


